In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import gfile

from collections import Counter
from itertools import combinations

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
## Lees csv-files in
csv_path = 'csv_files/'
mid_to_label_new = pd.read_csv(str(csv_path) + 'class_labels_indices_amsterdam.csv', quotechar = '"')
balanced_train_segments = pd.read_csv(str(csv_path) + 'balanced_train_segments.csv', sep = ";")
unbalanced_train_segments = pd.read_csv(str(csv_path) + 'unbalanced_train_segments.csv', sep = ";")

In [3]:
## Verkrijg alle labels en labelcombinaties (uit csv-file)
def getLabelOccurences(segments_file):
    
    ## Houd labels bij en label-combinaties (elke combi uit een example)
    labels = []
    label_combinations = []

    # for each row
    for i in range(segments_file.shape[0]):

        # labels uit de example
        label_combination = []
        mids = segments_file.loc[i, 'positive_labels'].split(",")
        
        # for each m-id in a string
        for mid in mids:
            new_label = mid_to_label_new.loc[mid_to_label_new['mid']==str(mid), 'index']
            
            if(new_label.shape[0]>0):
                labels.extend(new_label)
                new_label_combi = str(new_label.values[0])
                label_combination.extend(new_label_combi)
    
        if len(label_combination)>0:
            label_combinations.append(label_combination)
    
    return(labels, label_combinations)

In [5]:
## Kies of je de balanced of unbalanced set wilt lezen
#bal_labels, bal_label_combinations = getLabelOccurences(balanced_train_segments)
unbal_labels, unbal_label_combinations = getLabelOccurences(unbalanced_train_segments)

In [6]:
## Houd totale aantal bij
sum_occur = 0
all_labels = unbal_labels # bal_labels

## Laat # voorkomens per label zien
cnt_labels = Counter(all_labels)
for i in cnt_labels.most_common(len(all_labels)):
    sum_occur = sum_occur + i[1]
    print(mid_to_label_new.loc[np.int(i[0])]['display_name'], ' : ', str(i[1]))

Speech  :  999421
Music  :  999366
Vehicle  :  126411
Musical instrument  :  116405
Car  :  40978
Animal  :  39326
Domestic animals, pets  :  19082
Boat, Water vehicle  :  13090
Accelerating, revving, vroom  :  11827
Motor vehicle (road)  :  8891
Motor vehicle (road)  :  8891
Rail transport  :  8691
Tools  :  7906
Motorcycle  :  7140
Idling  :  6089
Medium engine (mid frequency)  :  6012
Emergency vehicle  :  5445
Aircraft  :  5254
Bus  :  5171
Heavy engine (low frequency)  :  4113
Gunshot, gunfire  :  3869
Car passing by  :  3684
Vehicle horn, car horn, honking  :  3587
Police car (siren)  :  3538
Power tool  :  3141
Fire engine, fire truck (siren)  :  3078
Conversation  :  2144
Ambulance (siren)  :  1805
Machine gun  :  1737
Chainsaw  :  1667
Traffic noise, roadway noise  :  1566
Engine starting  :  988
Hammer  :  385
Train wheels squealing  :  382
Jackhammer  :  161


In [7]:
label_combinations = unbal_label_combinations

## Bereken aantal voorkomens per combinatie
## Tellen van lijsten in lijsten is lastig, vandaar dat ik er strings van maak: [0,4] wordt ["0_4"]
str_combis = []

for combi in label_combinations:
    str_combi = [str(i) for i in combi]
    str_combi = "_".join(str_combi)
    str_combis.append(str_combi)

In [ ]:
## Geef meest voorkomende combinaties

cnt_label_combinations = Counter(str_combis)
for label_combination in cnt_label_combinations.most_common(20):
    label_names = []
    label_list = label_combination[0].split("_")
    for i in label_list:
        label_names.append(str(mid_to_label_new.loc[np.int(i)]['display_name']))
    label_names = ', '.join(label_names)
    print(label_names,': ', label_combination[1])

In [ ]:
## Code die kolom toevoegt aan label-csv (proportion) en deze in de forloop vult

mid_to_label_new["proportion"] = 0

for i in range(mid_to_label_new.shape[0]):
    nr_occur = cnt_labels[i]
    mid_to_label_new.loc[i,"proportion"] = round(nr_occur/sum_occur,8)

In [ ]:
## Schrijf bestandje weg
mid_to_label_new.to_csv(str(csv_path) + "class_labels_indices_proportions.csv", sep = ";")

In [ ]:
###############################################################################
####
###############################################################################
#### Hieronder staat andere code (deze code telt de labels vanuit tf-records)

In [11]:
## Folder met tf-records
train_data_pattern = 'tfrecords/train_all_data_new_temp/*.tfrecord'
test_data_pattern = 'tfrecords/test_data_new/*.tfrecord'

In [16]:
## Hieronder worden labels geteld door het lezen van tf-records
all_labels = []
label_combinations = []

In [17]:
def getLabelsFromTfRecord(tfrecords_filename):
    
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)
    
    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        ## Get labels
        new_labels = example.context.feature['labels'].int64_list.value
       
        for label in new_labels:
            all_labels.append(int(label))
        label_combinations.append(new_labels)

In [18]:
## Labels uit tf-records
files = gfile.Glob(train_data_pattern)

for file in files:
   
    getLabelsFromTfRecord(file)

In [19]:
## Houd totale aantal bij
sum_occur = 0

## Laat # voorkomens per label zien
cnt_labels = Counter(all_labels)
for i in cnt_labels.most_common(len(all_labels)):
    sum_occur = sum_occur + i[1]
    print(mid_to_label_new.loc[np.int(i[0])]['display_name'], ' : ', str(i[1]))

Music  :  383247
Speech  :  382078
Vehicle  :  48564
Musical instrument  :  45706
Car  :  16007
Animal  :  14847
Domestic animals, pets  :  7582
Boat, Water vehicle  :  4891
Accelerating, revving, vroom  :  4677
Motor vehicle (road)  :  3580
Rail transport  :  3136
Tools  :  2879
Medium engine (mid frequency)  :  2754
Idling  :  2718
Motorcycle  :  2497
Aircraft  :  1947
Emergency vehicle  :  1911
Bus  :  1852
Gunshot, gunfire  :  1639
Heavy engine (low frequency)  :  1607
Car passing by  :  1564
Vehicle horn, car horn, honking  :  1322
Police car (siren)  :  1202
Power tool  :  1154
Fire engine, fire truck (siren)  :  1096
Conversation  :  861
Ambulance (siren)  :  641
Machine gun  :  579
Chainsaw  :  576
Traffic noise, roadway noise  :  568
Engine starting  :  443
Gunshot SC9mm  :  206
Hammer  :  135
Train wheels squealing  :  128
Gunshot SC556  :  77
Jackhammer  :  60


In [20]:
#label_combinations = getLabelCombinations(balanced_train_segments)
#label_combinations = getLabelCombinations(unbalanced_train_segments)

## Bereken aantal voorkomens per combinatie
## Tellen van lijsten in lijsten is lastig, vandaar dat ik er strings van maak: [0,4] wordt ["0_4"]
str_combis = []

for combi in label_combinations:
    str_combi = [str(i) for i in combi]
    str_combi = "_".join(str_combi)
    str_combis.append(str_combi)

In [21]:
## Geef meest voorkomende combinaties

cnt_label_combinations = Counter(str_combis)
for label_combination in cnt_label_combinations.most_common(20):
    label_names = []
    label_list = label_combination[0].split("_")
    for i in label_list:
        label_names.append(str(mid_to_label_new.loc[np.int(i)]['display_name']))
    label_names = ', '.join(label_names)
    print(label_names,': ', label_combination[1])

Speech :  235200
Music :  219667
Speech, Music :  112806
Music, Musical instrument :  39481
Vehicle :  10143
Speech, Vehicle :  6463
Speech, Music, Musical instrument :  5332
Vehicle, Car :  5072
Animal :  3951
Speech, Animal :  3695
Animal, Domestic animals, pets :  3041
Speech, Vehicle, Car :  2808
Speech, Animal, Domestic animals, pets :  2623
Vehicle, Boat, Water vehicle :  1467
Vehicle, Car, Accelerating, revving, vroom :  1442
Speech, Vehicle, Boat, Water vehicle :  1375
Rail transport :  1308
Boat, Water vehicle :  1298
Vehicle, Motorcycle :  1274
Vehicle, Rail transport :  1270


In [22]:
sum_occur

944731